### **Segunda limpieza del dataset**

---

#### Importar excel (*01_ventas.xls*) para trabajar sobre los atributos desde Pandas


- VENTA/hora: **(”dia/mes/año HH:mm:ss” —— “HH:mm”)**

In [ ]:
# pip install xlrd
import pandas as pd

df_venta = pd.read_excel('01_ventas.xls', sheet_name="VENTA")
df_producto = pd.read_excel('01_ventas.xls', sheet_name="PRODUCTO")
df_detalle_venta = pd.read_excel('01_ventas.xls', sheet_name="DETALLE_VENTA")

print(df_producto.head())

   idProducto           nombre categoria  cantidad  total_ARS
0           1        Medialuna    Dulces      2487    3091300
1           2            Latte     Cafés      1412    5261410
2           3            Chipá   Salados      1382    3498725
3           4       Flat White     Cafés       933    3539800
4           5  Onda Medialunas    Promos       721    4103500


---

#### Reconfigurar PKs (*idProducto*) de PRODUCTO:
- Prefijo: primeras tres letras de la columna *categoria* en mayúscula.
- Sufijo: valor autoincremental (001, 002, ...) para cada producto dentro de la categoría.

In [ ]:
# Generar IDs de 3 cifras por cada productos dentro de una categoría
df_producto["id_nuevo"] = (
    df_producto.groupby("categoria").cumcount() + 1
).astype(str).str.zfill(3)

# Generar prefijo en mayúscula con las primeras 3 letras de la categoría
df_producto["id_nuevo"] = df_producto["categoria"].str[:3].str.upper() + df_producto["id_nuevo"]

# Cambiar los valores de idProducto por los nuevos
df_producto["idProducto"] = df_producto["id_nuevo"]
df_producto = df_producto.drop(columns=["id_nuevo"])

print(df_producto.head(10))

  idProducto             nombre categoria  cantidad  total_ARS
0     DUL001          Medialuna    Dulces      2487    3091300
1     CAF001              Latte     Cafés      1412    5261410
2     SAL001              Chipá   Salados      1382    3498725
3     CAF002         Flat White     Cafés       933    3539800
4     PRO001    Onda Medialunas    Promos       721    4103500
5     DUL002  Cookie Red Velvet    Dulces       643    2416600
6     DUL003     Cookie Clásica    Dulces       582    1964100
7     DUL004   Pan de Chocolate    Dulces       499    2179400
8     CAF003              Magic     Cafés       464    1463400
9     DUL005          Croissant    Dulces       464    1342600


---

#### Mapear *DETALLE_VENTA/producto* con *PRODUCTO/idProducto*:

- Eliminar información redundante.
- Relacionar ambas tablas.

In [ ]:
# Crear mapa de nombres a IDs
mapa = dict(zip(df_producto["nombre"], df_producto["idProducto"]))

# Reemplazar en detalle_venta
df_detalle_venta["idProducto"] = df_detalle_venta["producto"].map(mapa)

# Eliminar columnas reemplazadas
df_detalle_venta = df_detalle_venta.drop(columns=["producto", "categoria"])

# Ordenar columnas
df_detalle_venta = df_detalle_venta[["idVenta", "idProducto", "cantidad", "precio", "costo", "cancelada"]]

print(df_detalle_venta.head())

   idVenta idProducto  cantidad  precio    costo cancelada
0     6812     CAF001         1    4200  1155.47        No
1     6812     DUL004         1    5000  2250.00        No
2     6812     ADI001         1    1500   174.00        No
3     6811     DUL003         1    3800  1100.00        No
4     6811     DUL009         1    4500  1434.00        No
